In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

ecg=np.array([])
for i in range (1,3):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Ecg200(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
print('wdth=',wdth)
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('shape(ecg)=',np.shape(ecg))
os.chdir(cwd)
base_wn=np.array([])
windws=np.array([])

for cls in range (1,3):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

wdth= 481
shape(ecg)= (100, 481)
shape wndws1= (31, 481)
shape base_wn= (100, 481)
shape furir_windws0= (100, 480)
shape furir_windws= (100, 481)
smpl_rte= 480
mx_wndws= 69


In [3]:
path=('{}\AugEvl'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvl/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [4]:
xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,3):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'11']=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'21']=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'31']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'41']=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'51']=np.empty((0,smpl_rte+1), float)     
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)

'''f_scr=np.array([54,83,88,82,70,58,54,70])      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
f_scr_difrnc=f_scr-f_scr[0]
f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
aug_amnt=np.array(f_scr_rtio)                 #aug_amnt[0] is UnAg so in follow start from 1 for scale,..
'''
vlum_wndw=6*mx_wndws  #5000

for i in range (1,3):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=6*mx_wndws-len(vars()['wndws'+str(i)])
  
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.1),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    vars()['ecg_aug'+str(i)+'11']=np.append(vars()['ecg_aug'+str(i)+'11'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'11'],11,i)
    print(str(i)+'11 =',len(vars()['ecg_aug'+str(i)+'11']))
    #xtrain=np.append(xtrain,vars()['ecg_aug10'+str(i)],axis=0)
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    vars()['ecg_aug'+str(i)+'21']=np.append(vars()['ecg_aug'+str(i)+'21'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.1),axis=0) 
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'21'],21,i)
    print(str(i)+'21 =',len(vars()['ecg_aug'+str(i)+'21']))
    #xtrain=np.append(xtrain,vars()['ecg_aug21'+str(i)],axis=0)
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.1),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    vars()['ecg_aug'+str(i)+'31']=np.append(vars()['ecg_aug'+str(i)+'31'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'31'],31,i)
    print(str(i)+'31 =',len(vars()['ecg_aug'+str(i)+'31']))
    #xtrain=np.append(xtrain,vars()['ecg_aug30'+str(i)],axis=0)
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDORaw(ecg,i,int(vlum_sig),mag=.1),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    vars()['ecg_aug'+str(i)+'41']=np.append(vars()['ecg_aug'+str(i)+'41'],GDORaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'41'],41,i)
    print(str(i)+'41 =',len(vars()['ecg_aug'+str(i)+'41']))
    #xtrain=np.append(xtrain,vars()['ecg_aug40'+str(i)],axis=0)
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.1),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))
    vars()['ecg_aug'+str(i)+'51']=np.append(vars()['ecg_aug'+str(i)+'51'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.2),axis=0)   
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'51'],51,i)
    print(str(i)+'51 =',len(vars()['ecg_aug'+str(i)+'51']))
    #xtrain=np.append(xtrain,vars()['ecg_aug51'+str(i)],axis=0)'''

"for i in range (1,3):                           #تعریف آرایه ی پنجره های کلاس ها\n    vlum_sig=6*mx_wndws-len(vars()['wndws'+str(i)])\n  \n    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.1),axis=0)#0.1\n    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)\n    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))\n    vars()['ecg_aug'+str(i)+'11']=np.append(vars()['ecg_aug'+str(i)+'11'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)\n    addqlty(base_wn,vars()['ecg_aug'+str(i)+'11'],11,i)\n    print(str(i)+'11 =',len(vars()['ecg_aug'+str(i)+'11']))\n    #xtrain=np.append(xtrain,vars()['ecg_aug10'+str(i)],axis=0)\n    \n    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)\n    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)\n    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))\n    vars()['ecg_aug'+str(i)+'21']=np.append(vars()['ecg_aug'+str(i)+'21'],

In [5]:
for i in range (1,3):
    aug_pls60,aug_pls61=2,3 #2 is MgWrp and 3 is TimeWrp Augmentes Data for Training GAN
    vars()['ecg_aug'+str(i)+'60']=np.append(vars()['ecg_aug'+str(i)+'60'],GAN_Gnrt(ecg,i,aug_pls60),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'60'],60,i)
    print(str(i)+'60 =',len(vars()['ecg_aug'+str(i)+'60']))
    vars()['ecg_aug'+str(i)+'61']=np.append(vars()['ecg_aug'+str(i)+'61'],GAN_Gnrt(ecg,i,aug_pls61),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'61'],61,i)
    print(str(i)+'61 =',len(vars()['ecg_aug'+str(i)+'61']))
    #xtrain=np.append(xtrain,vars()['ecg_aug60'+str(i)],axis=0)
    
    aug_pls70,aug_pls71=2,3 #2 is MgWrp and 3 is TimeWrp Augmentes Data for LSTM Augmentation
    vars()['ecg_aug'+str(i)+'70']=np.append(vars()['ecg_aug'+str(i)+'70'],LSTM_Gnrt_Pridc(ecg,i,aug_pls70),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'70'],70,i)
    print(str(i)+'70 =',len(vars()['ecg_aug'+str(i)+'70']))
    vars()['ecg_aug'+str(i)+'71']=np.append(vars()['ecg_aug'+str(i)+'71'],LSTM_Gnrt_Pridc(ecg,i,aug_pls71),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'71'],71,i)
    print(str(i)+'71 =',len(vars()['ecg_aug'+str(i)+'71']))




 Class  1  Shape =  (100, 481)
for class  1 number of windows is =  (481,)
GAN Train Data shape= (138, 480)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Optimal cluster number:  3
class= 1 cluster= 0 epochs= 2172  >> shape generate =  (651600, 483)
class= 1 cluster= 1 epochs= 2172  >> shape generate =  (651600, 483)
class= 1 cluster= 2 epochs= 2172  >> shape generate =  (651600, 483)
cls  1  final shape, Generated by GAN = (690, 483)
shape and aug[0]= (481,) [2.4921875  2.48046875 2.50390625 2.53710938 2.55859375 2.59960938
 2.63476562 2.68554688 2.71679688 2.75       2.77734375 2.81445312
 2.84765625 2.87695312 2.91210938 2.94140625 2.96679688 2.98632812
 3.00976562 3.03125    3.05859375 3.08203125 3.11523438 3.14257812
 3.171875   3.19726562 3.23242188 3.25976562 3.28710938 3.31054688
 3.34179688 3.37304688 3.40234375 3.43359375 3.46875    3.49414062
 3.5234375  3.54296875 3.57226562 3.58398438 3.60546875 3.61523438
 3.62304688 3.62695312 3.6171875  3.59960938 3.578125   3.55273438
 3.50976562 3.45898438 3.40820312 3.359375   3.30664062 3.25585938
 3.19921875 3.14453125 3.07421875 3.01367188 2.94921875 2.89257812
 2.83203125 2.771484

class= 1 shape(new5)= (690, 486)	mx= 4.234375	mn= 1.0

161 = 690



 Class  2  Shape =  (100, 481)
for class  2 number of windows is =  (481,)
GAN Train Data shape= (138, 480)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Optimal cluster number:  4
class= 2 cluster= 0 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 1 epochs= 2630  >> shape generate =  (789000, 483)
class= 2 cluster= 2 epochs= 1368  >> shape generate =  (410400, 483)
class= 2 cluster= 3 epochs= 4000  >> shape generate =  (640000, 483)
cls  2  final shape, Generated by GAN = (690, 483)
shape and aug[0]= (481,) [3.16015625 3.34765625 3.53515625 3.78125    3.9609375  4.15625
 4.3046875  4.44921875 4.52734375 4.56640625 4.546875   4.484375
 4.359375   4.19140625 4.00390625 3.75976562 3.56640625 3.35742188
 3.14648438 2.94335938 2.79296875 2.66796875 2.58203125 2.5390625
 2.54296875 2.578125   2.65625    2.74414062 2.85546875 2.96875
 3.07421875 3.15820312 3.20507812 3.23828125 3.22851562 3.20507812
 3.17382812 3.15039062 3.13085938 3.1171875  3.11523438 3.12109375
 3.125      3.13085938 3.12304688 3.1015625  3.06640625 3.03125
 2.984375   2.93359375 2.87695312 2.8203125  2.76171875 2.71289062
 2.640625   2.58007812 2.49218

class= 2 shape(new5)= (690, 486)	mx= 4.67578125	mn= 1.013671875

260 = 690



 Class  2  Shape =  (100, 481)
for class  2 number of windows is =  (481,)
GAN Train Data shape= (138, 480)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Optimal cluster number:  3
class= 2 cluster= 0 epochs= 4000  >> shape generate =  (640000, 483)
class= 2 cluster= 1 epochs= 1110  >> shape generate =  (333000, 483)
class= 2 cluster= 2 epochs= 3030  >> shape generate =  (606000, 483)
cls  2  final shape, Generated by GAN = (690, 483)
shape and aug[0]= (481,) [3.2109375  3.69140625 3.60351562 3.50390625 3.43164062 3.22851562
 2.94140625 2.65429688 2.46289062 2.41796875 2.56445312 2.74804688
 2.90429688 3.04492188 3.14453125 3.13867188 3.09179688 3.04492188
 3.0234375  2.99414062 3.00195312 3.0390625  3.08398438 3.125
 3.15625    3.16015625 3.1484375  3.1171875  3.00976562 2.890625
 2.77734375 2.65039062 2.52929688 2.41601562 2.31445312 2.21875
 2.18164062 2.15625    2.15234375 2.1484375  2.16015625 2.15234375
 2.1640625  2.16210938 2.16992188 2.15429688 2.140625   2.11132812
 2.08789062 2.05859375 2.03320312 1.99804688 1.98632812 1.97070312
 1.96582031 1.95019531 1.95703125 1.94238281 1.95703125 1.95410156
 1.96972656 1.96484375 1.98144

class= 2 shape(new5)= (690, 486)	mx= 4.66796875	mn= 1.001953125

261 = 690
